In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
data = pd.read_csv('us-states.csv')

In [4]:
d = {
    'Alabama' : 95,
    'Alaska' : 1,
    'American Samoa' : 721,
    'Arizona' : 60,
    'Arkansas' : 57,
    'California' : 251,
    'Colorado' : 52,
    'Connecticut' : 741,
    'Delaware' : 485,
    'District of Columbia' : 11011,
    'Florida' : 378,
    'Georgia' : 177,
    'Guam' : 808,
    'Hawaii' : 222,
    'Idaho' : 20,
    'Illinois' : 231,
    'Indiana' : 184,
    'Iowa' : 55,
    'Kansas' : 36,
    'Kentucky' : 112,
    'Louisiana' : 108,
    'Maine' : 43,
    'Maryland' : 618,
    'Massachusetts' : 871,
    'Michigan' : 175,
    'Minnesota' : 68,
    'Mississippi' : 63,
    'Missouri' : 88,
    'Montana' : 7,
    'Nebraska' : 24,
    'Nevada' : 26,
    'New Hampshire' : 148,
    'New Jersey' : 1218,
    'New Mexico' : 17,
    'New York' : 420,
    'North Carolina' : 206,
    'North Dakota' : 10,
    'Northern Mariana Islands' : 307,
    'Ohio' : 284,
    'Oklahoma' : 57,
    'Oregon' : 41,
    'Pennsylvania' : 286,
    'Puerto Rico' : 1046,
    'Rhode Island' : 1021,
    'South Carolina' : 162,
    'South Dakota' : 11,
    'Tennessee' : 160,
    'Texas' : 105,
    'Utah' : 36,
    'Vermont' : 67,
    'Virgin Islands' : 799,
    'Virginia' : 212,
    'Washington' : 107,
    'West Virginia' : 76,
    'Wisconsin' : 106,
    'Wyoming' : 6
}

In [8]:
popSeries = pd.Series(d)
a = []
for x in range(0,2889):
    a.append(popSeries[data['state'].iloc[x]])
addSeries = pd.Series(a)

In [10]:
data = pd.concat([data, addSeries.reindex(data.index)], axis = 1)

In [13]:
data.columns = ['date', 'state', 'fips', 'cases', 'deaths', 'pop_density']

In [14]:
data.head()

,date,state,fips,cases,deaths,pop_density
0,2020-01-21,Washington,53,1,0,107
1,2020-01-22,Washington,53,1,0,107
2,2020-01-23,Washington,53,1,0,107
3,2020-01-24,Illinois,17,1,0,231
4,2020-01-24,Washington,53,1,0,107


In [15]:
data.shape

(2889, 6)

In [19]:
import datetime as dt 

In [21]:
#label encoder for dates
def scrub_date(date1):
    date_parts = date1.split('-')
    f_date = dt.date(int(date_parts[0]), int(date_parts[1]), int(date_parts[2]))
    delta = f_date - dt.date(2020, 1, 21)
    return delta.days

for i in range(len(data['date'])):
    data['date'][i] = scrub_date(data['date'][i])

/home/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [22]:
#label encoder for state
id = 1
state_dict = {}
for i in range(len(data['state'])):
    state = data['state'][i]
    if state not in state_dict:
        state_dict[state] = id
        id += 1
        data['state'][i] = state_dict[state]
    else:
        data['state'][i] = state_dict[state]
        

/home/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/justin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [23]:
data = data.drop('fips', axis = 1)

In [24]:
y = data.cases
X = data.drop('cases', axis = 1)

In [25]:
data

,date,state,cases,deaths,pop_density
0,0,1,1,0,107
1,1,1,1,0,107
2,2,1,1,0,107
3,3,2,1,0,231
4,3,1,1,0,107
...,...,...,...,...,...
2884,93,33,11000,375,212
2885,93,1,12906,717,107
2886,93,54,981,31,76
2887,93,6,5052,257,106


In [26]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [27]:
from sklearn.ensemble import RandomForestRegressor

In [28]:
model = RandomForestRegressor(n_estimators = 100, random_state = 1)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [29]:
preds = model.predict(X_valid)

In [30]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_valid, preds)

In [31]:
mae

437.62307053941896

In [32]:
data.describe()

,cases,deaths,pop_density
count,2889.000000,2889.00000,2889.000000
mean,4734.918311,190.28972,420.761855
std,18762.852274,967.45956,1407.884868
min,0.000000,0.00000,1.000000
25%,19.000000,0.00000,55.000000
50%,365.000000,6.00000,108.000000
75%,2331.000000,63.00000,284.000000
max,263460.000000,15740.00000,11011.000000


In [46]:
import tensorflow_probability as tfp

tfd = tfp.distributions

In [57]:
def build_tf_model():
    model = keras.Sequential([
        layers.Dense(64, activation = 'relu', input_shape=[len(X_train.keys())]),
        layers.Dense(64, activation = 'relu'),
        layers.Dense(64, activation = 'relu'),
        tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t, scale=1)),
        layers.Dense(1)
    ])
    
    #optimizer = tf.keras.optimizers.RMSprop(0.001)
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
    
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mae', 'mse'])
    
    return model 

tf_model = build_tf_model()

tf_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 64)                320       
_________________________________________________________________
dense_20 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_21 (Dense)             (None, 64)                4160      
_________________________________________________________________
distribution_lambda_6 (Distr ((None, 64), (None, 64))  0         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 65        
Total params: 8,705
Trainable params: 8,705
Non-trainable params: 0
_________________________________________________________________


In [58]:
history = tf_model.fit(X_train, y_train, epochs = 20, verbose = 1)

Train on 2166 samples
Epoch 1/20
2166/2166 [==============================] - 0s 224us/sample - loss: 294266396.7572 - mae: 4272.0093 - mse: 294266400.0000
Epoch 2/20
2166/2166 [==============================] - 0s 114us/sample - loss: 106003172.5199 - mae: 2851.6995 - mse: 106003168.0000
Epoch 3/20
2166/2166 [==============================] - 0s 159us/sample - loss: 13745705.9557 - mae: 1373.3236 - mse: 13745707.0000
Epoch 4/20
2166/2166 [==============================] - 0s 126us/sample - loss: 12394088.6400 - mae: 1088.8829 - mse: 12394088.0000
Epoch 5/20
2166/2166 [==============================] - 0s 144us/sample - loss: 12139752.1010 - mae: 1046.4485 - mse: 12139755.0000
Epoch 6/20
2166/2166 [==============================] - ETA: 0s - loss: 10612494.5754 - mae: 1025.9148 - mse: 10612495.00 - 0s 153us/sample - loss: 12233326.0665 - mae: 1069.1588 - mse: 12233326.0000
Epoch 7/20
2166/2166 [==============================] - 0s 171us/sample - loss: 12418995.9638 - mae: 1069.2523 - m

In [43]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,mae,mse,epoch
45,1.127258e+07,1053.259644,11272579.0,45
46,1.122976e+07,1052.183228,11229759.0,46
47,1.146790e+07,1067.820923,11467904.0,47
48,1.092086e+07,1032.137573,10920861.0,48
49,1.211319e+07,1117.964111,12113184.0,49


In [36]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [37]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std = 2)

In [39]:
plotter.plot({'Basic': history}, metric = "mae")
plt.ylabel('MAE [Confirmed Cases]')

KeyError: 'val_mae'